# The Task is - Pull data from web, convert to csv, upload to a data server, Execute SQL queries on the data

In [1]:
# Pull stock data from yahoo
# install required packages
#!pip install pandas_datareader

Import required packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as dr
from datetime import date
from datetime import timedelta

Initializing process. Creating list of stocks and seeting a date range (10 days) to extract data

In [5]:
# List of stocks
stocks = ['ADANIPORTS','ASIANPAINT','AXISBANK','BAJAJ-AUTO','BAJFINANCE','BAJAJFINSV','BPCL','CIPLA','COALINDIA','DRREDDY','EICHERMOT','GAIL','GRASIM','HCLTECH','HDFCBANK','HEROMOTOCO','HINDALCO','HINDPETRO','HINDUNILVR','HDFC','ITC','ICICIBANK','IBULHSGFIN','IOC','INDUSINDBK','INFY','KOTAKBANK','LT','LUPIN','M&M','MARUTI','NTPC','ONGC','POWERGRID','RELIANCE','SBIN','SUNPHARMA','TCS','TATAMOTORS']

# setting date range
today = date.today()
day_diff = timedelta(1)
end_dt = today - day_diff
day_diff = timedelta(10)
start_dt = today - day_diff

Looping through list of stock and pulling data for fixed date range (6 working days for each stock) from yahoo

In [6]:
df = pd.DataFrame(columns=['Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close','Stock'])

for stock in stocks:    
    df_stock = dr.DataReader(f"{stock}.NS", data_source = "yahoo",start=start_dt,end=end_dt)
    
    # Reseting index to default pandas index. In Raw data date was set as index
    
    df_stock2 = df_stock.reset_index(col_level=1)
    
    # creating new column 'Stock' which will contain the name of stock. It's not in raw data but will be useful for identification of stock
    
    df_stock2['Stock'] = stock
    df = df.append(df_stock2)
    

df.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Stock
0,2022-04-27,924.650024,880.000000,914.049988,887.299988,20793885,887.299988,ADANIPORTS
1,2022-04-28,900.250000,876.799988,899.000000,887.099976,8778304,887.099976,ADANIPORTS
2,2022-04-29,898.900024,852.099976,892.700012,856.400024,11543840,856.400024,ADANIPORTS
3,2022-05-02,862.849976,838.049988,850.000000,856.549988,5760499,856.549988,ADANIPORTS
4,2022-05-04,864.200012,805.799988,860.500000,812.799988,7969164,812.799988,ADANIPORTS


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 0 to 6
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       273 non-null    datetime64[ns]
 1   High       273 non-null    float64       
 2   Low        273 non-null    float64       
 3   Open       273 non-null    float64       
 4   Close      273 non-null    float64       
 5   Volume     273 non-null    object        
 6   Adj Close  273 non-null    float64       
 7   Stock      273 non-null    object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 19.2+ KB


Exporting dataframe as CSV to user-defined location

In [6]:
df.to_csv(r'/Users/arkadutta/Desktop/IIQF PGPAT/stock_data_test.csv',index=True, header=True)

SQL Part

In [9]:
#!pip install pymysql

In [10]:
#!pip install sqlalchemy

In [1]:
import pymysql
from sqlalchemy import create_engine

In [1]:
userid = 'Your user id as string'
pwd = 'Your password as string'
db = 'Your Database Name'
host = 'Host Name where you have created the db'


In [ ]:
engine = create_engine(f'mysql+pymysql://{userid}:{pwd}@{host}/{db}')

In [9]:
df.to_sql(con = engine, name = 'stock_data', if_exists= 'replace', index = True) # uploading table to DB

In [25]:
query = "SELECT DATE, STOCK , CLOSE, OPEN FROM stock_data"
df2 = pd.read_sql(query, engine)
df2.head()

,DATE,STOCK,CLOSE,OPEN
0,2022-04-25,ADANIPORTS,859.549988,873.950012
1,2022-04-26,ADANIPORTS,909.500000,868.000000
2,2022-04-27,ADANIPORTS,887.299988,914.049988
3,2022-04-28,ADANIPORTS,887.099976,899.000000
4,2022-04-29,ADANIPORTS,856.400024,892.700012


In [32]:
query = "select DATE, STOCK, CLOSE, OPEN, (CLOSE-OPEN)/OPEN as stock_return from stock_data"
df2 = pd.read_sql(query, engine)


In [33]:
df2.head()

,DATE,STOCK,CLOSE,OPEN,stock_return
0,2022-04-27,ADANIPORTS,887.299988,914.049988,-0.029265
1,2022-04-28,ADANIPORTS,887.099976,899.000000,-0.013237
2,2022-04-29,ADANIPORTS,856.400024,892.700012,-0.040663
3,2022-05-02,ADANIPORTS,856.549988,850.000000,0.007706
4,2022-05-04,ADANIPORTS,812.799988,860.500000,-0.055433


In [34]:
df2.to_sql(con = engine, name = 'stock_data2', if_exists= 'replace', index = True)

In [42]:
query = "select * from stock_data2 where stock_return in (select MAX(stock_return) as stock_return_max from stock_data2 group by DATE)"
df2 = pd.read_sql(query, engine)
df2

,index,DATE,STOCK,CLOSE,OPEN,stock_return
0,8,2022-04-28,ASIANPAINT,3247.699951,3190.000000,0.018088
1,105,2022-04-27,HEROMOTOCO,2500.000000,2390.000000,0.046025
2,110,2022-05-05,HEROMOTOCO,2501.649902,2430.100098,0.029443
3,111,2022-05-06,HEROMOTOCO,2563.350098,2410.000000,0.063631
4,171,2022-05-02,INDUSINDBK,1018.099976,972.000000,0.047428
5,184,2022-04-29,KOTAKBANK,1790.750000,1768.949951,0.012324
6,228,2022-05-04,ONGC,162.250000,159.000000,0.020440


In [49]:
query = "select count(distinct STOCK) as num_stocks, count(distinct DATE) as num_days from stock_data2"
pd.read_sql(query, engine)

,num_stocks,num_days
0,39,7
